In [321]:
pip install galois

Note: you may need to restart the kernel to use updated packages.


In [322]:
import galois
import numpy as np

In [323]:
#definisco un campo di Galois come nell'esempio 
field = galois.GF(2**3)

In [324]:
#definisco una matrice k x (n-k) dove n-k = 4
k = 3 #le righe
n_k = 4 #le colonne

In [325]:
A = field.Random((k, n_k))
print(A)

[[1 5 7 2]
 [4 0 2 2]
 [4 6 2 2]]


In [326]:
matrice_bin1 = np.array([
    [0, 0, 1],
    [1, 0, 0],
    [0, 1, 0]
])

matrice_bin1 = field(matrice_bin1)

matrice_bin2 = np.array([
    [1, 0, 0, 0],
    [0, 0, 1, 0],
    [0, 0, 0, 1],
    [0, 1, 0, 0]
])

matrice_bin2 = field(matrice_bin2)

In [327]:
A_primo_intermedio = field(np.dot(matrice_bin1, A))
print(A_primo_intermedio)

[[4 6 2 2]
 [1 5 7 2]
 [4 0 2 2]]


In [328]:
A_primo = field(np.dot(A_primo_intermedio, matrice_bin2))
print(A_primo)

[[4 2 6 2]
 [1 2 5 7]
 [4 2 0 2]]


In [329]:
import threading

In [330]:
#qui in accordo con slide 18 articolo devo ordinare la matrice A e A_primo per righe e colonne, ordinamento per riga 
#è dato dalla somma delle righe in ordine decrescente, mentre per colonna in ordine numerico del primo elemento e in caso di 
#parità del secondo e così via
print(type(A))

#lista per conservare somme delle righe, inizializzata con gli zeri
somme_rig = [0] * A_primo.shape[0]
primi_colonne = [0] * A_primo.shape[1]
secondi_colonne = [0] * A_primo.shape[1]

#creo il lock per evitare conglitti nell'aggiornamento
lock = threading.Lock()

#funzione per somma elementi nella colonna
def somma_riga(matrice, indice_riga):
    riga = matrice[indice_riga, :]
    print(riga) #questi vanno eliminati poi
    somma = np.sum(riga)
    print(somma) #questi vanno eliminati poi
    with lock: #vado ad usare il lock
        somme_rig[indice_riga] = somma
        print(somme_rig)

def estrai_da_colonna(matrice, indice_colonna):
    colonna  = matrice[:, indice_colonna]
    with lock:
        primi_colonne[indice_colonna] = colonna[0]
        secondi_colonne[indice_colonna] = colonna[1]


<class 'galois.GF(2^3)'>


In [331]:
threads_righe = []
threds_colonne = []
colonne_finali = []

In [332]:
#creo un thread per ogni colonna
for indice_riga in range(A_primo.shape[0]):
    t = threading.Thread(target = somma_riga, args = (A_primo, indice_riga))
    print(t)
    threads_righe.append(t)
    t.start() #avvio il thread

#aspetto il risultato di tutti i threads
for t in threads_righe:
    t.join()


<Thread(Thread-195 (somma_riga), initial)>
[4 2 6 2]
2
[GF(2, order=2^3), 0, 0]
<Thread(Thread-196 (somma_riga), initial)>
[1 2 5 7]
1
[GF(2, order=2^3), GF(1, order=2^3), 0]
<Thread(Thread-197 (somma_riga), initial)>
[4 2 0 2]
4
[GF(2, order=2^3), GF(1, order=2^3), GF(4, order=2^3)]


In [333]:
print ("\nSomma degli elementi di ciascuna riga: ", somme_rig)
print ("\n Elementi primi estratti: ", primi_colonne)
print ("\n Elementi secondi estratti: ", secondi_colonne)


Somma degli elementi di ciascuna riga:  [GF(2, order=2^3), GF(1, order=2^3), GF(4, order=2^3)]

 Elementi primi estratti:  [0, 0, 0, 0]

 Elementi secondi estratti:  [0, 0, 0, 0]


In [334]:
indici_ordinati = np.argsort(somme_rig)
A_canonica_righe = A_primo[indici_ordinati, :]

print(A_canonica_righe)

[[1 2 5 7]
 [4 2 6 2]
 [4 2 0 2]]


In [335]:
for indice_colonna in range(A_primo.shape[1]):
    p = threading.Thread(target = estrai_da_colonna, args = (A_canonica_righe, indice_colonna))
    threds_colonne.append(p)
    p.start()

for p in threds_colonne:
    p.join()

indici_ordinati_col = np.argsort(primi_colonne)


for idx in indici_ordinati_col:
    colonne_finali.append((primi_colonne[idx], secondi_colonne[idx], idx))

colonne_finali.sort(key=lambda x: (x[0], x[1]))

A_CF = A_canonica_righe[:, [col[2] for col in colonne_finali]]

print(A_CF)

[[1 2 5 7]
 [4 2 6 2]
 [4 2 0 2]]
